## TO DO:
- modified pamguard in table to be a list of lists at init

In [91]:
from math import pi
import math
from datetime import datetime 

from config import AMP_heading
from config import M3_swath
from config import M3_avgeraging_time
from targets import TargetSpace
from targets import Target
import rules
import comms
import classification as classif

In [101]:
# add to classification class __init__:
# self.classified_targets_count = 0
classified_targets_count= 15

# add these to config
refit_classifier_count = 10
drop_target_time = 30 

In [99]:
def update_target_space(classifier, ts):
    """
    Refit classifier and drop old targets from target space.
    """
    if classified_targets_count >= refit_classifier_count:
        # refit
        # X = add new targets to current model targets
        # radius_neighbors_instance.fit(X)
        pass
    
# TODO: do we want to remove old targets every iteration? or only on refit?
# TODO: ask bernease, how to remove classifier features??
# TODO: will every target with None in agg indices be handled?
# TODO: ask bernease, is newest ADCP data always last?

    min_time = ts.tables['nims'][0][0]
    min_time = remove_old_nims(ts, min_time)
    min_time = remove_old_pamguard(ts, min_time)  
    remove_old_adcp(ts, min_time)
    

def remove_old_nims(ts, min_time):
    """
    Remove nims data older than drop_target_time. Update minimum
    time of all targets (to avoid dropping relevant ADCP data)
    """
    # remove all targets with nims that have not been seen 
    # for drop_target_time seconds
    classification_ind = 0
    
    for i, target in enumerate(ts.tables['nims']):
        if target[0] < min_time:
            min_time = target[0]

        if target[-1] and delta_t_in_seconds(datetime.now(), target[0]) >= drop_target_time:
            target[-1].append(i)

            for index in sorted(target[-1], reverse = True):
                del ts.tables['nims'][index] 

            del ts.tables['classifier_classifications'][classification_ind]
            classification_ind += 1
    
    return(min_time)


def remove_old_pamguard(ts, min_time):
    """
    Remove pamguard data older than drop_target_time. Update minimum
    time of all targets (to avoid dropping relevant ADCP data)
    """
    for i, target in reversed(list(enumerate(ts.tables['pamguard']))):
        if delta_t_in_seconds(datetime.now(), target[0]) >= drop_target_time:
            del ts.tables['pamguard'][i]

            if target[0] < min_time:
                min_time = target[0]
    
    return min_time


def remove_old_adcp(ts, min_time):
    # remove all adcp data older than minimum time, except for the most recent
    # entry
    for i, adcp in reversed(list(enumerate(ts.tables['adcp']))):
        if adcp[0] < min_time and i < (len(ts.tables['adcp'])-1):
            del ts.tables['adcp'][i]


# this function is already in code...import?
def delta_t_in_seconds(datetime1, datetime2):
    """
    calculate delta t in seconds between two datetime objects
    (returns absolute value, so order of dates is insignifigant)
    """
    delta_t = datetime1 - datetime2
    days_s = delta_t.days*(86400)
    microseconds_s = delta_t.microseconds/1000000
    delta_t_s = days_s + delta_t.seconds + microseconds_s
    return abs(delta_t_s)
                    
        

# test

In [100]:
ts = reinit_ts()
print('length of nims table before removing old data: ',len(ts.tables['nims']))
print('length of pam table before removing old data: ', len(ts.tables['pamguard']))
print('length of adcp table before removing old data: ', len(ts.tables['adcp']))

remove_old_targets(classif, ts)

print()
print('length of nims table after removing old data: ', len(ts.tables['nims']))
print('length of pam table after removing old data: ', len(ts.tables['pamguard']))
print('length of adcp table after removing old data: ', len(ts.tables['adcp']))

length of nims table before removing old data:  11
length of pam table before removing old data:  1
length of adcp table before removing old data:  2

length of nims table after removing old data:  8
length of pam table after removing old data:  0
length of adcp table after removing old data:  1


# sample target space

In [94]:
def reinit_ts():
    ts = TargetSpace()

    ts.tables = {'adcp': [[datetime(2016, 3, 8, 23, 47, 16, 600000),1.2,4.5],
      [datetime(2016, 3, 8, 23, 47, 16, 600000),5.6,2.5]],
     'camera': [],
     'classifier_classifications': [1],
     'classifier_features': [],
     'nims': [[datetime(2016, 3, 8, 23, 47, 16, 606170),
       213,
       1225,
       26648,
       3.255762865359059,
       1.009538904487781,
       0.33651296816259363,
       0.3397229332247963,
       1,
       0,
       0,
       120,
       50,
       81.59374223255094,
       38.3912730413624,
       None],
      [datetime(2016, 3, 8, 23, 47, 16, 606170),
       221,
       764,
       27109,
       2.6585455797134,
       0.9990226307834469,
       0.3330075435944823,
       0.3326820722724931,
       4.749735623188873e-05,
       0,
       0,
       120,
       50,
       46.89115349587096,
       36.90071673015692,
       None],
      [datetime(2016, 3, 8, 23, 47, 16, 606170),
       230,
       662,
       27211,
       -1.4252885390060634,
       3.144248887280707,
       1.0480829624269024,
       3.2954336883886546,
       1,
       0,
       0,
       120,
       50,
       64.45498648235723,
       45.6643834242734,
       None],
      [datetime(2016, 3, 8, 23, 47, 16, 606170),
       232,
       532,
       27341,
       2.890572359393626,
       3.632297183140556,
       1.2107657277135186,
       4.397860942216939,
       1,
       0,
       0,
       120,
       50,
       62.32347596039271,
       36.98009428240289,
       None],
      [datetime(2016, 3, 8, 23, 47, 16, 606170),
       233,
       243,
       27630,
       0.765336539780397,
       2.0736340537675213,
       0.6912113512558404,
       1.4333193963147746,
       0.006207837847612895,
       0,
       0,
       120,
       50,
       49.21580297831671,
       23.907033796827886,
       None],
      [datetime(2016, 3, 8, 23, 47, 18, 880554),
       213,
       1226,
       26648,
       3.327882881606094,
       1.009538904487781,
       0.33651296816259363,
       0.3397229332247963,
       0.04768479173654866,
       0,
       0,
       120,
       50,
       82.17312165722745,
       38.70562258528113,
       None],
      [datetime(2016, 3, 8, 23, 47, 16, 880554),
       221,
       765,
       27109,
       2.561834802661609,
       0.9990226307834469,
       0.3330075435944823,
       0.3326820722724931,
       4.2568959617736036e-06,
       0,
       0,
       120,
       50,
       46.89112643708125,
       36.90070054109199,
       None],
      [datetime(2016, 3, 8, 23, 47, 16, 880554),
       230,
       663,
       27211,
       -1.347039165042165,
       3.144248887280707,
       1.0480829624269024,
       3.2954336883886546,
       1,
       0,
       0,
       120,
       50,
       65.05607597023428,
       45.51592097830661,
       None],
      [datetime(2016, 3, 8, 23, 47, 16, 880554),
       232,
       533,
       27341,
       2.844387888261456,
       3.632297183140556,
       1.2107657277135186,
       4.397860942216939,
       0.06967176481633819,
       0,
       0,
       120,
       50,
       61.548294381352996,
       36.95444344388781,
       None],
      [datetime(2016, 3, 8, 23, 47, 16, 880554),
       233,
       244,
       27630,
       0.7439338175799398,
       2.0736340537675213,
       0.6912113512558404,
       1.4333193963147746,
       1,
       0,
       0,
       120,
       50,
       49.22192287980031,
       23.90526033283612,
       None],
     [datetime(2016, 3, 8, 23, 47, 16, 880554),
        218,
        244,
        27630,
        0.7439338175799398,
        2.0736340537675213,
        0.6912113512558404,
        1.4333193963147746,
        1,
        0,
        0,
        120,
        50,
        49.22192287980031,
        23.90526033283612,
        [0,5]]],
     'pamguard': [[datetime(2016, 3, 8, 23, 47, 16, 800000),"Dolphin"]]}
    
    return ts

In [56]:
len(ts.tables['nims'])

11